# Timeseries module

In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

from clouddrift import ragged_array
from clouddrift import timeseries

## Create a ragged array for the demonstration

In [ ]:
from data import gdp

drifter_ids = gdp.download(n_random_id=100)

# keep only required variables (from coords, metadata, and data)
coords = {'ids': 'ids', 'time': 'time', 'lon': 'longitude', 'lat': 'latitude'}
metadata = ['rowsize']
data = ['ve', 'vn']

ra = ragged_array.from_files(drifter_ids,
                         gdp.preprocess, # function of the dataset
                         coords, 
                         metadata, 
                         data,
                         rowsize_func=gdp.rowsize
                        )

ds = ra.to_awkward()  # return an Awkward Array

In [ ]:
%%time
ret = timeseries.periodogram(ds.obs.ve + 1j*ds.obs.vn)

In [ ]:
# selected trajectory [0,100]
i = 42

# calculate inertial frequency in cycle per day [cpd] from the mean latitude of trajectory
omega = 7.2921159e-5  # Earth's rotation rate [rad/s]
seconds_per_day = 60*60*24  # [s]
fi = -2*omega*(seconds_per_day/(2*np.pi))*np.sin(np.mean(np.radians(ds.obs.lat[i])))

# define frequency scale/abscissa
dt = 1/24  # [-]
f = np.fft.fftfreq(ds.rowsize[i], dt) 
ss = np.log10(ret[i])  # spectrum of the ith trajectory 

fig = plt.figure(dpi=100)
ax = fig.add_subplot(1,1,1)

h1, = ax.plot(np.fft.fftshift(f), np.fft.fftshift(ss), scaley=True, label='Spectrum', linewidth=0.5)
h2 = ax.axvline(x=fi,color='k',linestyle=':', label='Inertial frequency', linewidth=0.5)
ax.legend(frameon=False)
ax.set_ylabel('PSD (m$^2$ s$^{-2}$ cpd$^{-1}$)')
ax.set_xlabel('Frequency [cpd]');